In [1]:
#Librerias
import joblib

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import StandardScaler # for standardization us

In [2]:
# Carga de datos
data = pd.read_csv("example_reduced.csv")

In [13]:
data.shape

(49999, 4)

In [4]:
X = data["text_cleaned"]
y = data["class"]

In [6]:
# Tokenization is necesary for preprocessing text to ML
# We will use CountVectorizer

vect = CountVectorizer()
X_vectorized = vect.fit_transform(X) 

In [7]:
# Standardization
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_vectorized)

### prueba del modelo Xgboost con el dataframe de prueba

In [8]:
# Cargar el modelo desde el archivo .pkl
xgboost_model_loaded = joblib.load('xgboost_model.pkl')

In [11]:
print(X_vectorized.shape)  # Debe ser (n_samples, 416525)


(49999, 50452)


In [9]:
y_pred_prueba = xgboost_model_loaded.predict(X_scaled)

ValueError: Feature shape mismatch, expected: 416525, got 50452

In [ ]:
# Evaluación con etiquetas verdaderas
y_true_prueba = df_prueba['class']  # Obtener las etiquetas verdaderas
accuracy_prueba = accuracy_score(y_true_prueba, y_pred_prueba)
print("Precisión del modelo en el dataframe de prueba:", accuracy_prueba)
print("\nInforme de clasificación para el dataframe de prueba:\n", classification_report(y_true_prueba, y_pred_prueba))